# Personalize에서 보류 세트에 대한 시간축성 평가 사용

이 노트북은 대부분 기본 노트북을 따르며, 모든 사용자에 대해 "미래" 데이터의 10%를 추가로 보유합니다. 그런 다음, 보류된 데이터에 대해 추천을 제공하고 외부에서 평가하기 위해 추론 엔드포인트를 설정합니다.

다른 사소한 차이점은 평점이 아니라 조회수를 예측한다는 것입니다. (개인화할 필요 없이) 모든 사람이 즐길 수 있는 인기 영화를 추천하는 것보다, 인기 없는(하지만 개인화 수준이 높은) 영화의 추천을 제시하는 것이 더 흥미롭다고 생각합니다.

In [ ]:
import boto3, os
import json
import numpy as np
import pandas as pd
import time
from botocore.exceptions import ClientError
!pip install tqdm
from tqdm import tqdm_notebook
from metrics import mean_reciprocal_rank, ndcg_at_k, precision_at_k

In [ ]:
suffix = str(np.random.uniform())[4:9]

In [ ]:
bucket = "demo-temporal-holdout-"+   suffix        # replace with the name of your S3 bucket
filename = "DEMO-temporal-holdout.csv"

In [ ]:
!aws s3 mb s3://{bucket}

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# 벤치마크 데이터 다운로드 및 처리

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip
data = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=['USER_ID','ITEM_ID','EVENT_VALUE', 'TIMESTAMP'])

In [ ]:
pd.set_option('display.max_rows', 5)

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

### 사용자당 마지막 10%의 상호 작용을 보류 테스트로 추출

In [ ]:
ranks = data.groupby('USER_ID').TIMESTAMP.rank(pct=True, method='first')
data = data.join((ranks>0.9).to_frame('holdout'))
holdout = data[data['holdout']].drop('holdout', axis=1)
data = data[~data['holdout']].drop('holdout', axis=1)

In [ ]:
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

### 데이터 업로드

In [ ]:
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

# 스키마 생성

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-temporal-schema-"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## 데이터 세트 및 데이터 세트 그룹

### 데이터 세트 그룹 생성

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-temporal-dataset-group-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

### ‘상호 작용’ 데이터 유형 생성

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "DEMO-temporal-dataset-"+suffix
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Personalize 액세스를 위한 S3 버킷 권한

### S3 버킷에 정책 연결

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

### 올바른 권한을 가진 역할 생성

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

# 데이터 세트 가져오기 작업 생성

상호 작용 데이터 업로드

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-temporal-dataset-import-job-"+suffix,
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

### 데이터 세트 가져오기 작업 및 데이터 세트 가져오기 작업 실행이 활성 상태가 될 때까지 기다리기

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

# 솔루션 생성

In [ ]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

다양한 시나리오를 지원하는 여러 가지 레시피가 있습니다. 이 예제에서는 상호 작용 데이터만 있으므로 기본 레시피 중 하나를 선택합니다.

| 현실성 | 레시피 | 설명 
|-------- | -------- |:------------
| 예 | aws-popularity-count | 사용자 항목 상호 작용 데이터 세트에서 해당 항목에 대한 이벤트 수를 기준으로 항목의 인기를 계산합니다.
| 예 | aws-hrnn | 사용자가 상호 작용할 항목을 예측합니다. 사용자 항목 상호 작용의 시간 순서를 모델링할 수 있는 계층적 반복 신경망입니다.
| 아니요 - 메타데이터 필요 | aws-hrnn-metadata | 사용자가 상호 작용할 항목을 예측합니다. 상황별 메타데이터(사용자-항목 상호 작용 메타데이터), 사용자 메타데이터(사용자 데이터 세트) 및 항목 메타데이터(항목 데이터 세트)에서 파생된 추가 특성이 있는 HRNN입니다.
| 아니요 - 밴디트(bandit)의 경우, 메타데이터 필요 | aws-hrnn-coldstart | 사용자가 상호 작용할 항목을 예측합니다. 새로운 항목에 대한 개인화된 탐색이 있는 HRNN 메타데이터입니다.
| 아니요 - 항목 기반 쿼리의 경우 | aws-sims | 사용자-항목 상호 작용 데이터 세트의 동일한 사용자 기록에서 항목의 동시 발생 사례를 기준으로 특정 항목과 유사한 항목을 계산합니다.
| N - 짧은 목록의 순위를 다시 매기는 경우 | aws-personalized-ranking | 사용자에 맞게 항목 목록을 다시 정렬합니다. 사용자-항목 데이터 세트에 대해 훈련합니다.


수동으로 또는 AutoML을 통해 이러한 기본 레시피를 모두 실행하고 내부 지표에서 가장 효과적인 모델을 선택할 수 있습니다. 개인화를 통해 지표가 개선되는 것을 확인하려면 특히 인기 기준과 비교해 보는 것이 좋습니다. 하지만 이 데모에서는 외부 평가에 초점을 맞추기 위해 aws-hrnn이라는 레시피를 하나 선택합니다.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

### 솔루션 생성 및 기다리기
이 작업은 2단계 프로세스로 이루어집니다.
1. 솔루션 생성
2. 솔루션 버전 생성

In [ ]:
create_solution_response = personalize.create_solution(
    name = "DEMO-temporal-solution-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

### 솔루션 버전이 활성 상태가 될 때까지 기다리기

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 솔루션의 지표 가져오기

In [ ]:
get_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_metrics_response, indent=2))

# 캠페인 생성 및 기다리기

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-temporal-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

### 캠페인이 활성 상태가 될 때까지 기다리기

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

# 외부 지표를 사용하여 평가

평가 지표에 대한 설명은 https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html 에서 확인할 수 있습니다.

예를 들어 4개의 항목을 추천하는데, 그중 2개가 $r=[0,1,0,1]$로 연관성이 있다고 가정해 보겠습니다. 이 예에서 지표는 다음과 같습니다.

|이름	|예	|설명
|:------|:----------|:----------
|Precision@K	|$\frac{2}{4} = 0.5$	|총 연관 항목 수를 총 추천 항목 수로 나눈 값입니다.
|평균 상호 순위(MRR@K)	|${\rm mean}(\frac{1}{2} + \frac{1}{4}) = 0.375$	|모든 연관 항목의 역방향 위치 평균을 계산하여 위치 효과를 반영합니다.
|정규화된 절감 누적 이득(NDCG@K)	|$\frac{\frac{1}{\log(1 + 2)} + \frac{1}{\log(1 + 4)}}{\frac{1}{\log(1 + 1)} + \frac{1}{\log(1 + 2)}} = 0.65$	|이상적인 추천에서 가능한 최대 점수로 정규화된 연관 항목의 위치를 기준으로, 역로그 가중치를 적용하여 위치 효과를 반영합니다.
|평균 정밀도(AP@K)	|${\rm mean}(\frac{1}{2} + \frac{2}{4}) = 0.5$	|precision@K의 평균입니다. 여기서 K는 모든 연관 항목의 위치입니다.

이들 지표는 두 가지 측면에서 내부 지표와 다릅니다.
* 서로 다른 시간에 평가되므로 클릭률을 다를 수 있습니다. 시간축성 드리프트를 방지하기 위해 항상 평가를 같은 시간으로 유지하는 것이 좋습니다.
* 외부 평가의 예제에서는 여러 항목을 보류하고 실제 항목으로 간주할 수 있는 반면, 내부 평가에서는 각 사용자 기록의 마지막 항목만 실제 항목으로 간주할 수 있습니다. 보류해야 하는 항목 수에 대해 절대적으로 선호되는 값은 없습니다. 실제 사용 사례와 유사한 평가 방법을 설계하는 것이 좋습니다.

In [ ]:
relevance = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    relevance.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))

### 선택 사항: 이전 구매 기록을 중복 제거하면 약간 더 나은 결과를 얻을 수 있습니다.

In [ ]:
rel_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    topk = [int(x['itemId']) for x in rec_response['itemList']]
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    

    rel_dedup.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_dedup]))

### 인기 기준을 더미 추천 시스템으로 삼아 비교해 보세요.

In [ ]:
topk = data.groupby("ITEM_ID").TIMESTAMP.count().sort_values(ascending=False).iloc[:100].index.values

In [ ]:
rel_popular = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_items = topk[:25]
    rel_popular.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_popular]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_popular]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_popular]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_popular]))

### 인기 기준에는 사용자 기록이 중복되어 있습니다.

In [ ]:
rel_pop_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    
    rel_pop_dedup.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_pop_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_pop_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_pop_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_pop_dedup]))

# 정리

In [ ]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)

personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

# 개인용 버킷을 사용하는 경우 이 셀을 주의하여 실행하세요.

In [ ]:
!aws s3 rm s3://{bucket} --recursive